In [1]:
import requests
from pymongo import MongoClient
import pandas as pd
import random

# Create user

In [2]:
personajes = ['Barack Obama', 'Voldemort', 'Chewbacca', 'Angela Merkel','Margherita Hack','Valentino Rossi', 
              'must4in3', 'Aretha Franklin', 'Marge Simpson', 'Darth Vader', 'Yoda', 'R2-D2', 'John Belushi',
              'Maccio Capatonda', ' Elon Musk', 'Che Guevara', 'Gollum', 'Samwise Gamgee', 'Jon Snow','Arya Stark',
              'Heidi', 'Hermione', 'Pikachu', 'Topolino', 'Berlusconi', 'Greta Thunberg','Hulk', 'Dilan Dog', 
              'Stephen Hopkins','Oprah Winfre', 'Frida Kahlo','Tyrion Lannister', 'Jennifer Aniston']

for personage in personajes:
    res = requests.get(f'http://localhost:3000/user/create/{personage}')

In [3]:
print(res)

<Response [200]>


In [4]:
res.json()

{'data': {'_id': {'$oid': '5f0e0cadbd0cd626e25ed252'},
  'username': 'Jennifer Aniston'},
 'status': 'ok'}

In [5]:
# check users in Mongo
dbName = "api_chat_sentiment_analysis"
mongodbURL = f"mongodb://localhost/{dbName}"
client = MongoClient(mongodbURL, connectTimeoutMS=2000, serverSelectionTimeoutMS=2000)
db = client.get_database()

In [6]:
cur = db.user.find({"username":{"$ne": None}})

In [7]:
df = pd.DataFrame(cur)
df.head()

,_id,username
0,5f0e0cadbd0cd626e25ed232,Barack Obama
1,5f0e0cadbd0cd626e25ed233,Voldemort
2,5f0e0cadbd0cd626e25ed234,Chewbacca
3,5f0e0cadbd0cd626e25ed235,Angela Merkel
4,5f0e0cadbd0cd626e25ed236,Margherita Hack


# Create Chat

In [8]:
df["_id"]= df["_id"].astype(str) 

In [9]:
five_chats = ['How I met your mother', 'Dumbledore’s Army', 
              'Tesla SpaceX','Italian restaurants in Madrid','datamad0620']
chats_room = dict()
for i,chat in enumerate(five_chats):   
    chats_room[f'{chat}']= {'user_ids': [random.choice([row for row in df['_id']]) for _ in range(5)]}

In [10]:
for room,user_ids in chats_room.items():
    res = requests.get(f'http://localhost:3000/chat/create/{room}?user_id={user_ids.values()}')

In [11]:
print(res)

<Response [200]>


In [12]:
res.json()

{'_id': {'$oid': '5f0e0cb0bd0cd626e25ed257'},
 'chat_name': 'datamad0620',
 'users_ids': ['5f0e0cadbd0cd626e25ed239',
  '5f0e0cadbd0cd626e25ed24e',
  '5f0e0cadbd0cd626e25ed24d',
  '5f0e0cadbd0cd626e25ed252',
  '5f0e0cadbd0cd626e25ed233']}

In [13]:
cur = db.chatItem.find({"chat_name":{"$ne": None}})

In [14]:
df_chat = pd.DataFrame(cur)
df_chat.head()

,_id,chat_name,users_ids
0,5f0e0cb0bd0cd626e25ed253,How I met your mother,"[5f0e0cadbd0cd626e25ed24e, 5f0e0cadbd0cd626e25..."
1,5f0e0cb0bd0cd626e25ed254,Dumbledore’s Army,"[5f0e0cadbd0cd626e25ed235, 5f0e0cadbd0cd626e25..."
2,5f0e0cb0bd0cd626e25ed255,Tesla SpaceX,"[5f0e0cadbd0cd626e25ed24f, 5f0e0cadbd0cd626e25..."
3,5f0e0cb0bd0cd626e25ed256,Italian restaurants in Madrid,"[5f0e0cadbd0cd626e25ed234, 5f0e0cadbd0cd626e25..."
4,5f0e0cb0bd0cd626e25ed257,datamad0620,"[5f0e0cadbd0cd626e25ed239, 5f0e0cadbd0cd626e25..."


# Add user in a Chat

In [45]:
df_chat["_id"]= df_chat["_id"].astype(str) 

In [46]:
user_random = random.choice([row for row in df['_id']])
conversation_random = random.choice([row for row in df_chat['_id']])

In [47]:
res = requests.get(f'http://localhost:3000/chat/{conversation_random}/adduser?user_id={user_random}')

In [48]:
print(res)

<Response [200]>


In [49]:
res.json()

{'chat_name': 'datamad0620',
 'users_ids': ['5f0e0cadbd0cd626e25ed239',
  '5f0e0cadbd0cd626e25ed24e',
  '5f0e0cadbd0cd626e25ed24d',
  '5f0e0cadbd0cd626e25ed252',
  '5f0e0cadbd0cd626e25ed233',
  '5f0e0cadbd0cd626e25ed243']}

# Add message in chat

kaggle dataset quote
https://www.kaggle.com/coolcoder22/quotes-dataset

In [50]:
quote = pd.read_csv('inputs/quote.csv', encoding = 'latin-1')

In [51]:
quote

,Auther,quote
0,A. A. Milne,"If you live to be a hundred, I want to live to..."
1,A. A. Milne,Promise me you'll always remember: You're brav...
2,A. A. Milne,"Did you ever stop to think, and forget to star..."
3,A. A. Milne,Organizing is what you do before you do someth...
4,A. A. Milne,"Weeds are flowers too, once you get to know them."
...,...,...
36160,Zsa Zsa Gabor,"I know nothing about sex, because I was always..."
36161,Zsa Zsa Gabor,Conrad Hilton was very generous to me in the d...
36162,Zsa Zsa Gabor,The women's movement hasn't changed my sex lif...
36163,Zsa Zsa Gabor,I always liked parties. You meet people; you c...


In [107]:
conversations = []
for _ in range(100):
    conversations.append({random.choice([row for row in df['_id']]): random.choice([row for row in quote['quote']])})

In [108]:
conversation_random = random.choice([row for row in df_chat['_id']])

In [109]:
for dic in conversations:
    for key,values in dic.items():
        res = requests.get(f'http://localhost:3000/chat/{conversation_random}/addmessage?user_id={key}&text={values}')

In [110]:
print(res)

<Response [200]>


In [111]:
res.json()

{'conversation_id': '5f0e0cb0bd0cd626e25ed256',
 'messages': 'We never fail when we try to do our duty, we always fail when we neglect to do it.'}

# Get messages list from a chat

In [112]:
conversation_random = random.choice([row for row in df_chat['_id']])

In [113]:
res = requests.get(f'http://localhost:3000/chat/{conversation_random}/list')

In [114]:
print(res)

<Response [200]>


In [115]:
res.json()

{'chat_name': 'datamad0620',
 'conversation_id': {'$oid': '5f0e0cb0bd0cd626e25ed257'},
 'messages': [{'message': 'The creative process is a process of surrender, not control.',
   'user_id': '5f0e0cadbd0cd626e25ed24d',
   'username': 'Dilan Dog'},
  {'message': 'Be careful to leave your sons well instructed rather than rich, for the hopes of the instructed are better than the wealth of the ignorant.',
   'user_id': '5f0e0cadbd0cd626e25ed233',
   'username': 'Voldemort'},
  {'message': "It's great that ballroom dancing is being recognised. For many years ballroom dancers were misunderstood and other dance forms didn't want anything to do with us.",
   'user_id': '5f0e0cadbd0cd626e25ed24d',
   'username': 'Dilan Dog'},
  {'message': 'The meeting of two personalities is like the contact of two chemical substances: if there is any reaction, both are transformed.',
   'user_id': '5f0e0cadbd0cd626e25ed239',
   'username': 'Aretha Franklin'},
  {'message': 'When you recognize the festive and 

# Get sentiment analysis

NLP & Text Sentiment Analysis:
​
- [https://www.nltk.org/]
- [https://towardsdatascience.com/basic-binary-sentiment-analysis-using-nltk-c94ba17ae386]
- [https://www.digitalocean.com/community/tutorials/how-to-perform-sentiment-analysis-in-python-3-using-the-natural-language-toolkit-nltk]

In [116]:
conversation_random = random.choice([row for row in df_chat['_id']])

In [117]:
res = requests.get(f'http://localhost:3000/chat/{conversation_random}/sentiment')

In [118]:
res.json()

[{'sentence': 'Curiosity is the wick in the candle of learning.',
  'sentiment': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}},
 {'sentence': 'I believe more in the scissors than I do in the pencil.',
  'sentiment': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}},
 {'sentence': 'The topic of leadership is a touchy one.',
  'sentiment': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}},
 {'sentence': "A lot of leaders fail because they don't have the bravery to touch that nerve or strike that chord.",
  'sentiment': {'neg': 0.337, 'neu': 0.663, 'pos': 0.0, 'compound': -0.7669}},
 {'sentence': "Throughout my years, I haven't had that fear.",
  'sentiment': {'neg': 0.0, 'neu': 0.695, 'pos': 0.305, 'compound': 0.3875}},
 {'sentence': "I feel like I'm held more accountable to stay healthy now because now I'm a role model to young girls to not have eating issues and to not say, 'Hey, it's OK to starve yourself' or 'It's OK to throw up after your meals' - that's not OK.",

# Get reccomender analysis

In [119]:
user_random = random.choice([row for row in df['_id']])

In [120]:
res = requests.get(f'http://localhost:3000/user/{user_random}/recommend')

In [121]:
print(res)

<Response [200]>


In [122]:
res.json()

{'Me': {'Che Guevara': 1.0},
 'Recommended users': {'Jon Snow': 0.692808772657237,
  'Marge Simpson': 0.7113370087114855,
  'Samwise Gamgee': 0.7273007448617727,
  'Tyrion Lannister': 0.6987606953986196,
  'must4in3': 0.6815839678381815}}